In [1]:
#there are many ways to get outta a maze
#this time, lets apply dijkstra and a* algo to the maze problem
#cuz we intend to get outta there asap
#otherwise, we can use bfs or dfs or other algos

#details of graph adt can be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb
class graph:
    def __init__(self):
        self.graph={}
        self.visited={}
       
    def append(self,vertexid,edge,weight):
        if vertexid not in self.graph.keys():
            self.graph[vertexid]={}
            self.visited[vertexid]=0
        self.graph[vertexid][edge]=weight

    def reveal(self):
        return self.graph

    def vertex(self):
        return list(self.graph.keys())

    def edge(self,vertexid):
        return list(self.graph[vertexid].keys())

    def weight(self,vertexid,edge):
        return (self.graph[vertexid][edge])

    def size(self):
        return len(self.graph)

    def visit(self,vertexid):
        self.visited[vertexid]=1

    def go(self,vertexid):
        return self.visited[vertexid]

    def route(self):
        return self.visited



In [2]:
#dijkstra is a special case of a*
#in dijkstra, heuristic is zero
#details of dijkstra algo could be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/dijkstra%20shortest%20path.ipynb

#the more general case, a* intends to find the shortest path too
#besides dijkstra distance, it has one more thing to calculate
#which is called heuristic
#it is a prediction of the steps to the end
#there are three different ways to calculate that

#the one we are gonna use is called manhattan distance
#because we can only move up, down, left, right in the maze
#think of the maze as street blocks in manhanttan
#the only way for taxi from one point to another is to drive through twists and turns
#denote start as (x,y), end as (i,j)
#the heuristic for manhattan distance is (np.abs(x-i)+np.abs(y-j))
#there are other two heuristic calculation
#one is called diagonal distance
#in order to use that, we must be able to move like a queen on the chessboard
#we should have eight different directions to move
#the heuristic for diagonal distance is max(np.abs(x-i),np.abs(y-j))
#another one is called euclidean distance
#it is like calculating the longest side of a triangle
#in order to use that, we must be able to move whatever directions we want
#every angle is valid for one step of move
#the heuristic for euclidean distance is ((x-i)**2+(y-j)**2)**0.5

#for a* search, we calculate both dijkstra distance and heuristic
#we get the sum of both for each edge of a vertex
#we append vertices with the minimum sum to the queue 
#we keep doing this just as dijkstra
#as we have two more variables, manhattan distance and sum of both distance
#a* would increase the space complexity
#however, it trades space for time complexity
#it is supposed to travel fewer vertices
#which means it should be faster than dijkstra
def astar(df,start,end):
    queue={}
    distance={}
    heuristic={}
    #route is a dict of the sum of distance and heuristic
    route={}
    queue[start]=0
    pred={}

    for i in df.vertex():
        distance[i]=float('inf')
        #manhattan distance
        heuristic[i]=np.abs(i[0]-end[0])+np.abs(i[1]-end[1])

    distance[start]=0 
    #k is to keep track of how many vertices we have traveled
    c=0    

    while queue:
        temp=min(queue,key=queue.get)
        queue.pop(temp)
        #minimum is to get the minimum sum of both distances
        minimum=float('inf')

        for j in df.edge(temp):
            distance[j]=distance[temp]+df.weight(temp,j)
            route[j]=distance[j]+heuristic[j]
            if route[j]<minimum:
                minimum=route[j]

        for j in df.edge(temp):
            #we only append unvisited and unqueued vertices
            #note that we could have two vertices with the minimum sum
            #that is why we use a loop to make sure all valid vertices are appended
            if (route[j]==minimum) and (df.go(j)==0) and (j not in queue):
                queue[j]=route[j]
                pred[j]=temp
                

        df.visit(temp)
        
        if temp==end:
                 break
        
        c+=1
    
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    print('vertice travelled:',c)
    return distance[end],path


#
def dijkstra(df,start,end):
    queue={}
    distance={}
    queue[start]=0
    pred={}
    c=0

    for i in df.vertex():
        distance[i]=float('inf')
    distance[start]=0    
        
    while queue:
        temp=min(queue,key=queue.get)
        queue.pop(temp)
        for j in df.edge(temp):
            if distance[temp]+df.weight(temp,j)<distance[j]:
                distance[j]=distance[temp]+df.weight(temp,j)
                pred[j]=temp
                
            if df.go(j)==0 and j not in queue:
                queue[j]=distance[j]
            
            
        df.visit(temp)
        if temp==end:
            break
        
        c+=1
    
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
     
    print('vertice travelled:',c)
    return distance[end],path

In [3]:
#the difficult part of a maze problem is to convert maze into a graph adt
#assume maze is a i*j matrix
#we can convert it into a tree or called list of list
#lets assign coordinates to each nodes in the tree
#the top left would be {0,0}
#the bottom right would be (i-1,j-1)

#this check_around function is designated to check nodes
#for a node with coordinate (i,j)
#we wanna check its upper node, lower node, left node and right node
#which are (i-1,j),(i+1,j),(i,j-1),(i,j+1)
#if those nodes mentioned above are not walls ('+' is wall in the maze)
#we append those edges into the graph
#but for boundary nodes, we wont find all four nodes near them
#therefore, we need to exclude index error
def check_around(i,j):
    for k,l in [(i-1,j),(i+1,j),(i,j-1),(i,j+1)]:
        try:
            if maze[k][l]!='+':
                mazerun.append((i,j),(k,l),1)
            
        except IndexError:
            pass

In [4]:
#read the maze file and print
#it can be downloaded from
# https://github.com/je-suis-tm/graph-theory/blob/master/data/maze.txt
import copy
import numpy as np
import os
os.chdir('d:/')
os.getcwd()
import re
f=open('maze.txt','r')
m=f.readlines()
for i in m:
    print(i)

++++++++++++++++++++++

+   +   ++ ++        +

      +     ++++++++++

+ +    ++  ++++ +++ ++

+ +   + + ++    +++  +

+          ++  ++  + +

+++++ + +      ++  + +

+++++ +++  + +  ++   +

+          + + S+ +  +

+++++ +  + + +     + +

++++++++++++++++++++++


In [5]:
#first, lets turn this txt file into a tree
#which is list of list
maze=[]
for i in m:
    maze.append(re.findall('.',i.replace('\n','')))

In [6]:
#for nodes in our maze tree
#S is the starting point
#when we find s, we set the start
#and dont forget to append start into graph adt
#when we find ' ', space is the valid path we can take
#we use check_around function to add edges
#when we find +, it is wall
#if there is a breach on the boundary wall
#we consider it as exit

#note that we assume there is only one exit in this maze
#we can also change the terminal to a point inside a maze
#in that case we have to consider the situation 
#what if we cannot get to the destination
mazerun=graph()
for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j]=='S':
            start=(i,j)
            check_around(i,j)
        elif maze[i][j]==' ':
            check_around(i,j)
            if (i==0) or (i==len(maze)-1) or (j==0) or (j==len(maze[0])-1):
                end=(i,j)
        else:
            pass

In [7]:
mazerun2=copy.deepcopy(mazerun)
mazerun3=copy.deepcopy(mazerun)
mazerun4=copy.deepcopy(mazerun)

In [8]:
#a* traveled fewer vertices than dijkstra
#cuz a* use heuristic to skip more vertices
#thus,a* is faster
#it is a trade off for time and space complexity
import datetime as dt
t1=dt.datetime.now()
print(astar(mazerun,start,end))
t2=dt.datetime.now()
print('astar:',(t2-t1).microseconds)

t1=dt.datetime.now()
print(dijkstra(mazerun2,start,end))
t2=dt.datetime.now()
print('dijkstra:',(t2-t1).microseconds)

vertice travelled: 44
(21, [(8, 15), (7, 15), (7, 14), (6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (5, 10), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0)])
astar: 0
vertice travelled: 85
(21, [(8, 15), (7, 15), (7, 14), (6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (5, 10), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0)])
dijkstra: 0


In [9]:
#details of bfs and dfs can be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb
#to make life easier, we use dfs iteration instead of dfs recursion
#we can make a small modification here
#so that bfs,dfs would give us the optimal path, the length of the path and how many vertices they have traveled
#the idea is the same as a*
#unlike the previous bfs, dfs, we stop the loop when we reach the destination

def bfs(df,start,end):
    queue=[]
    queue.append(start)
    pred={}
    c=0
    
    while queue:
        temp=queue.pop(0)
        df.visit(temp)
        for newpos in df.edge(temp):
            if df.go(newpos)==0 and newpos not in queue:
                queue.append(newpos)
                pred[newpos]=temp
                
        
                
        if temp==end:
            break
        
        c+=1
        
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    return len(path)-1,path

def dfs_itr(df,start,end):
    queue=[]
    queue.append(start)
    pred={}
    c=0
    
    while queue:
        temp=queue.pop(0)
        smallq=[]
        df.visit(temp)
        for newpos in df.edge(temp):
            if df.go(newpos)==0:
                if newpos in queue:
                    queue.remove(newpos)
                smallq.append(newpos)
                pred[newpos]=temp
                
        queue=smallq+queue
        
        if temp==end:
            break
        
        c+=1
        
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    return len(path)-1,path

In [10]:
#note that bfs is guaranteed to find the optimal path
#dfs may find the path, it could be very costly

t1=dt.datetime.now()
print(bfs(mazerun3,start,end))
t2=dt.datetime.now()
print('bfs:',(t2-t1).microseconds)

t1=dt.datetime.now()
print(dfs_itr(mazerun4,start,end))
t2=dt.datetime.now()
print('dfs:',(t2-t1).microseconds)

(21, [(8, 15), (7, 15), (7, 14), (6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (5, 10), (5, 9), (5, 8), (5, 7), (5, 6), (5, 5), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0)])
bfs: 0
(35, [(8, 15), (7, 15), (7, 14), (6, 14), (5, 14), (4, 14), (4, 13), (5, 13), (6, 13), (6, 12), (6, 11), (6, 10), (5, 10), (5, 9), (4, 9), (3, 9), (2, 9), (2, 8), (2, 7), (1, 7), (1, 6), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (5, 4), (4, 4), (3, 4), (2, 4), (2, 3), (1, 3), (1, 2), (2, 2), (2, 1), (2, 0)])
dfs: 0
